In [71]:
import simple_salesforce
import pandas as pd
import requests
import numpy as np
import datetime
import glob
import os
from simple_salesforce import Salesforce

In [65]:

chapter='Seattle'

In [107]:
sf = Salesforce(username=os.environ['SALESFORCE_EMAIL'],
                password=os.environ['SALESFORCE_PASSWORD'],
                security_token=os.environ['SALESFORCE_TOKEN'],
                instance='https://na88.lightning.force.com/',
                version='42.0')

In [162]:
salesforce_cleaning = {
    'Gates Foundation':'Bill & Melinda Gates Foundation',
    'Russell': 'Russell Investments',
    'Symetra': 'Symetra Financial',
    'Oracle': 'Oracle Corporation',
    'Expedia': 'Expedia Inc.'
}

In [78]:
report_files = glob.glob('../DonationReports/*.csv')

In [86]:
report_files[0]

'../DonationReports/DonationReport_AGSMYDRNP0_20200912.csv'

In [163]:
import urllib

def encode_dict(d):
    return {k: urllib.parse.quote(v) for k,v in d.items()}

def process_report(report_df):

    for k,row in report_df.iterrows():
        first_name = row['Donor First Name']
        last_name = row['Donor Last Name']
        donation = row['Total Donation to be Acknowledged']
        match = row['Match Amount']
        donation_date=datetime.datetime.strptime(row['Donation Date'], '%Y-%m-%dT%H:%M:%SZ')
        if first_name == "Not shared by donor":
            first_name = "Anonymous"
            last_name = "Gift"
        if (match>0):
            match_type = 'Employee Giving'
            match_name=f"${match} {row.Company} donation match of {first_name} {last_name} via Benevity on {donation_date.strftime('%Y-%m-%d')}"
        if (donation>0) & (match==0):
            match = donation
            donation=0
            match_type = 'Volunteer Grant'
            match_name=f"${match} {row.Company} volunteer hours match of {first_name} {last_name} via Benevity on {donation_date.strftime('%Y-%m-%d')}"

        new_first = first_name.replace("'", "%5C%27")
        new_last = last_name.replace("'", "%5C%27")
        qry_str = f"SELECT Id, Email FROM Contact WHERE FirstName = '{new_first}' AND LastName = '{new_last}'"
        records=sf.query(qry_str)
        if records['totalSize']==0:
            records=sf.query(f"SELECT Id, Email FROM Contact WHERE Email = '{row['Email']}' or npe01__WorkEmail__c = '{row['Email']}'")        
        if records['totalSize']==0:
            account_data= {
                'Name': f"{last_name} ({first_name})" + " Household",
                'npo02__Formal_Greeting__c': "{} {}".format(first_name,last_name),
                'npo02__Informal_Greeting__c': "{}".format(first_name),
                'RecordTypeId':'0121Y000001QTlbQAG'
            }
            if row['Address']!= 'Not shared by donor':
                account_data['BillingStreet']=row['Address']
                account_data['BillingCity']=row['City']
                account_data['BillingState']=row['State/Province']
                account_data['BillingPostalCode']=row['Postal Code']
            
            account_results=sf.Account.create(account_data)
            assert(account_results['success'])
            email = row['Email']
            if email == 'Not shared by donor':
                email=None
            contact_data={
                'FirstName': first_name,
                'LastName': last_name,
                'Email': email,
                'RecordTypeId': "012i0000000plOsAAI",
                'Chapter__c':chapter,
                'AccountId':account_results['id']
                #'AccountRecordType':'0121Y000001QTlbQAG', 
            }
            contact_results=sf.Contact.create(contact_data)
            assert(contact_results['success'])
            contactID=contact_results['id']
            accountId=account_results['id']
            print(f"needed to create account and contact record for {first_name} {last_name}")

        else:
            contactrecords=records['records']

            contactID=contactrecords[0]['Id']
            contact = sf.Contact.get(contactID)
            accountId = contact['AccountId']
            account = sf.Account.get(accountId)

        companyrecords = sf.query(f"SELECT Id FROM Account WHERE Name = '{row.Company}'")

        if companyrecords['totalSize']==0:
            print(f'company not found: {row.Company}')
            print("Need to create Account with this name")
            companyAccountId = None
        else:
            companyAccountId = companyrecords['records'][0]['Id']
            contactRoleData = None
            donation_data = {
                'RecordTypeId': '012i0000000oqevAAA',
                'Amount': donation,
                'AccountId': accountId,
                'StageName': 'Received',
                'Chapter__c': chapter,
                'Probability': 100.0,
                'CloseDate': donation_date.strftime('%Y-%m-%d'),
                'Type': 'Individual',
                'Check__c': row['Transaction ID'],
                'Name': f"{first_name} {last_name} ${donation} donation via Benevity on {donation_date.strftime('%Y-%m-%d')}"
            }
            if match>0:
                match_data = {
                    'RecordTypeId': '012i0000000oqevAAA',
                    'Amount': match,
                    'AccountId': companyAccountId,
                    'StageName': 'Received',
                    'Chapter__c': chapter,
                    'Probability': 100.0,
                    'CloseDate': donation_date.strftime('%Y-%m-%d'),
                    'Type': match_type,
                    'Check__c': row['Transaction ID'],
                    'Name': match_name
                }
                contactRoleData = {
                    "Role": "Soft Credit",
                    "ContactId": contactID,
                    "OpportunityId": "opportunityId",
                    "IsPrimary":False
                }

            already_imported = sf.query(f"SELECT Id FROM Opportunity WHERE Check__c = '{row['Transaction ID']}'")    
            if already_imported['totalSize']==0:
                print('importing row')
                if donation>0:
                    donation_result=sf.Opportunity.create(donation_data)
                    print(donation_result)
                if match>0:
                    print(match_data)
                    match_result=sf.Opportunity.create(match_data)
                    assert(match_result['success'])
                    contactRoleData["OpportunityId"]=match_result['id']
                    contactrole_result=sf.OpportunityContactRole.create(contactRoleData)
                    print(match_result, contactrole_result)
            else:
                print('skipping row already imported')


In [165]:

# a set of company renames that are necessary to map fields between Salesforce and Benevity systems.
for report_file in report_files:
    report_df = pd.read_csv(report_file, skiprows=10, skipfooter=4,thousands=',')
    report_df['Company']=report_df.Company.replace(salesforce_cleaning)
    process_report(report_df)

/opt/anaconda3/envs/mindsmatter/lib/python3.7/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
importing row
OrderedDict([('id', '0061Y00000tEZKbQAO'), ('success', True), ('errors', [])])
{'RecordTypeId': '012i0000000oqevAAA', 'Amount': 17.0, 'AccountId': '0011Y000039ImdVQAS', 'StageName': 'Received', 'Chapter__c': 'Seattle', 'Probability': 100.0, 'CloseDate': '2018-09-12', 'Type': 'Employee Giving', 'Check__c': 'A0TV27TY6C', 'Name': '$17.0 Keurig Dr Pepper donation match of Irina Baek via Benevity on 2018-09-12'}
OrderedDict([('id', '0061Y00000tEZKgQAO'), ('success', True), ('errors', [])]) OrderedDict([('id', '00K1Y00000LzH6iUAF'), ('success', True), ('errors', [])])
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already import

skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping r

skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping row already imported
skipping r

In [156]:
report_df.loc[2]['Donor First Name']=="Not shared by donor"

True

In [159]:
next(i for i,f in enumerate(report_files) if f == report_file)

30